In [17]:
import torch
import torchvision.transforms as tvt
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import ConcatDataset, DataLoader
from torchmetrics import AUROC  # additional dependency
from torchvision.datasets import MNIST, FashionMNIST, KMNIST

from oodtk import NegativeEnergy, Softmax
from oodtk.dataset.img import MNISTC, UniformNoise, GaussianNoise
from oodtk.model import WideResNet
from oodtk.utils import is_unknown, ToUnknown

In [18]:
torch.manual_seed(123)

trans = tvt.Compose([tvt.Resize((32,32)), tvt.ToTensor()])

# setup data
dataset_train = MNIST(root="data", train=True, download=True, transform=trans)
train_loader = DataLoader(dataset_train, batch_size=128)

In [19]:
model = WideResNet(num_classes=10, in_channels=1)
opti = Adam(model.parameters())
criterion = CrossEntropyLoss()
model = model.cuda()

In [20]:
model.train()

# start training
for epoch in range(10):
    for n, batch in enumerate(train_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        logits = model(x)
        loss = criterion(logits, y)
        opti.zero_grad()
        loss.backward()
        opti.step()


In [24]:
# create some methods
energy = NegativeEnergy(model).cuda()
softmax = Softmax(model).cuda()

In [26]:
dataset_in_test = MNIST(root="data", train=False, transform=trans)
dataset_out_test = MNISTC(root="data", subset="all", split="test", download=True,
                          transform=trans, target_transform=ToUnknown())
dataset_test = dataset_in_test + dataset_out_test
test_loader = DataLoader(dataset_test, batch_size=128)

# evaluate
auroc_energy = AUROC(num_classes=2)
auroc_softmax = AUROC(num_classes=2)
model.eval()

with torch.no_grad():
    for n, batch in enumerate(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        auroc_energy.update(energy(x), is_unknown(y))
        auroc_softmax.update(softmax(x), is_unknown(y))

print(auroc_softmax.compute())
print(auroc_energy.compute())

tensor(0.7762, device='cuda:0')
tensor(0.7042, device='cuda:0')


In [27]:
dataset_in_test = MNIST(root="data", train=False, transform=trans)
dataset_out_test = FashionMNIST(root="data", download=True, transform=trans, target_transform=ToUnknown())
dataset_test = dataset_in_test + dataset_out_test
test_loader = DataLoader(dataset_test, batch_size=128)

# evaluate
auroc_energy = AUROC(num_classes=2)
auroc_softmax = AUROC(num_classes=2)
model.eval()

with torch.no_grad():
    for n, batch in enumerate(test_loader):
        x, y = batch
        x = x.cuda()
        y = y.cuda()

        auroc_energy.update(energy(x), is_unknown(y))
        auroc_softmax.update(softmax(x), is_unknown(y))

print(auroc_softmax.compute())
print(auroc_energy.compute())

tensor(0.9550, device='cuda:0')
tensor(0.9327, device='cuda:0')
